In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.regularizers import l2

# Create generators to load image data

In [3]:
train_data = tf.keras.utils.image_dataset_from_directory(
    directory='Data_2/train/',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
)

val_data = tf.keras.utils.image_dataset_from_directory(
    directory='Data_2/test/',
    labels='inferred',
    label_mode='int',
    batch_size=32,
    image_size=(256, 256),
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.


## Normalize data

In [4]:
def preprocess(image, label):
    image = tf.cast(image/255.0, tf.float32)
    return image, label

In [5]:
train_data = train_data.map(preprocess)
val_data = val_data.map(preprocess)

## Model

In [8]:
model = Sequential()

## Convolutional layers
model.add(Conv2D(filters=32, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', input_shape=(256, 256, 3), name='2d_conv_l1'))
model.add(BatchNormalization(name='batch_normalization_l1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l1'))

model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', name='2d_conv_l2'))
model.add(BatchNormalization(name='batch_normalization_l2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l2'))

model.add(Conv2D(filters=128, kernel_size=(3, 3), strides=1, padding='valid', activation='relu', name='2d_conv_l3'))
model.add(BatchNormalization(name='batch_normalization_l3'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', name='max_pooling_l3'))

## Flatten Layer
model.add(Flatten(name='flatten'))

## Fully connected layers
model.add(Dense(units=128, activation='relu', kernel_regularizer=l2, name='fully_connected_layer_1'))
model.add(BatchNormalization(name='batch_normalization_l4'))
model.add(Dropout(0.1, name='Dropout_l1'))
model.add(Dense(units=64, activation='relu', kernel_regularizer=l2, name='fully_connected_layer_2'))
model.add(BatchNormalization(name='batch_normalization_l5'))
model.add(Dense(units=1, activation='sigmoid', kernel_regularizer=l2, name='fully_connected_layer_3'))
model.add(Dropout(0.1, name='Dropout_l2'))

model.name = 'cat_dog_cls'

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "cat_dog_cls"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ 2d_conv_l1 (Conv2D)             │ (None, 254, 254, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l1          │ (None, 254, 254, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l1 (MaxPooling2D)   │ (None, 127, 127, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 2d_conv_l2 (Conv2D)             │ (None, 125, 125, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l2          │ (None, 125, 125, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l2 (MaxPooling2D)   │ (None, 62, 62, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ 2d_conv_l3 (Conv2D)             │ (None, 60, 60, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l3          │ (None, 60, 60, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling_l3 (MaxPooling2D)   │ (None, 30, 30, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 115200)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_1 (Dense) │ (None, 128)            │    14,745,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l4          │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_l1 (Dropout)            │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_2 (Dense) │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_l5          │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fully_connected_layer_3 (Dense) │ (None, 1)              │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout_l2 (Dropout)            │ (None, 1)              │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,848,961 (56.64 MB)

 Trainable params: 14,848,129 (56.64 MB)

 Non-trainable params: 832 (3.25 KB)

## Train the model

In [9]:
history = model.fit(train_data, epochs=10, validation_data=val_data).history
model.save('cat_dog_cls')

Epoch 1/10
406/625 ━━━━━━━━━━━━━━━━━━━━ 5:01 1s/step - accuracy: 0.6019 - loss: 4.8535

KeyboardInterrupt: 

## Plot learning curve

In [ ]:
plt.plot(history['loss'], label='train loss', color='red')
plt.plot(history['val_loss'], label='val loss', color='green')
plt.title('Loss')
plt.legend()
plt.show()

plt.plot(history['accuracy'], label='train accuracy', color='red')
plt.plot(history['val_accuracy'], label='val accuracy', color='green')
plt.title('Accuracy')
plt.legend()
plt.show()